In [84]:
import pandas as pd
import pickle
import numpy as np

In [85]:
with open('df_to_model_1_3751.pickle', 'rb') as f:
    project_df = pickle.load(f)

In [86]:
project_df.head()

,kickstarter_page,description_length,n_pledges,pledge_level_min,pledge_level_max,pledge_level_stddev,funding_actual,funding_goal,n_backers,start_date,...,Film & Video,Food,Games,Journalism,Music,Photography,Publishing,Sculpture,Technology,Theater
0,/projects/1063028485/good-art-guide/,93,1,7.886086,7.886086,0.000000,0.0,15772.172704,0,2014-11-26,...,0,0,0,0,0,0,0,0,0,0
1,/projects/spiderwebcitytx/katlynn-maries-horro...,106,3,2.000000,100.000000,45.507020,17.0,15000.000000,3,2014-11-24,...,0,0,0,0,0,0,0,0,0,0
2,/projects/mnassar/wwwparkinglocatorcomau/,46,1,8.712913,8.712913,0.000000,0.0,43564.564775,0,2014-11-21,...,0,0,0,0,0,0,0,0,1,0
3,/projects/1934089488/not-enough-fish/,160,7,7.837804,783.780404,299.100883,0.0,1567.560808,0,2014-11-19,...,1,0,0,0,0,0,0,0,0,0
4,/projects/timodwyer/hurt-fate-feature-film/,153,10,1.564151,7820.753536,2342.290924,0.0,93849.042433,0,2014-11-17,...,1,0,0,0,0,0,0,0,0,0


In [87]:
project_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55808 entries, 0 to 1
Data columns (total 26 columns):
kickstarter_page       55808 non-null object
description_length     55808 non-null int64
n_pledges              55808 non-null int64
pledge_level_min       55808 non-null float64
pledge_level_max       55808 non-null float64
pledge_level_stddev    55808 non-null float64
funding_actual         55808 non-null float64
funding_goal           55808 non-null float64
n_backers              55808 non-null object
start_date             55808 non-null object
duration               55808 non-null int64
Comics                 55808 non-null uint8
Crafts                 55808 non-null uint8
Dance                  55808 non-null uint8
Design                 55808 non-null uint8
Fashion                55808 non-null uint8
Film & Video           55808 non-null uint8
Food                   55808 non-null uint8
Games                  55808 non-null uint8
Journalism             55808 non-null uint8
Mu

In [88]:
import matplotlib.pyplot as plt
%matplotlib inline

In [89]:
subset_df = project_df.drop(columns=['kickstarter_page', 'start_date'])

In [90]:
col_types = {'n_backers': int}
subset_df = subset_df.astype(col_types)

In [91]:
subset_df['funding'] = subset_df.funding_actual / subset_df.funding_goal
subset_df.funding = subset_df.funding.apply(lambda x: 1 if x>=1.0 else 0)
#subset_df['pledge_min_percent'] = subset_df.pledge_level_min / subset_df.funding_goal
subset_df.drop(columns=['funding_actual'], inplace=True)
subset_df.funding.value_counts()

0    36917
1    18891
Name: funding, dtype: int64

In [92]:
X = subset_df.drop(columns=['funding'])
y = subset_df.funding

In [93]:
from sklearn.model_selection import train_test_split

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [95]:
X_train.head()

,description_length,n_pledges,pledge_level_min,pledge_level_max,pledge_level_stddev,funding_goal,n_backers,duration,Comics,Crafts,...,Film & Video,Food,Games,Journalism,Music,Photography,Publishing,Sculpture,Technology,Theater
11,3218,20,5.000000,2000.000000,478.459661,50000.000000,1712,35,0,0,...,0,0,1,0,0,0,0,0,0,0
11,340,5,5.000000,2200.000000,843.625509,17925.000000,6,19,0,0,...,0,0,0,0,0,0,0,0,0,0
6,836,7,8.550386,1710.077127,570.664304,6840.308509,440,30,0,0,...,0,0,0,0,0,0,0,0,1,0
8,22,4,0.930735,9.307353,3.290646,18.614707,0,26,0,0,...,0,1,0,0,0,0,0,0,0,0
7,0,1,1.863231,1.863231,0.000000,93161.573249,0,30,0,0,...,0,0,0,0,0,0,0,0,1,0


In [96]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_predicted = logreg.predict(X_test)
logreg.score(X_test,y_test)

0.8945230842740249

In [97]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_predicted)
print(confusion_matrix)

[[10679   404]
 [ 1362  4298]]


In [98]:
def mean_squared_error(true, pred):
    return np.mean((pred - true)**2)

def root_mean_square_error(true,pred):
    return np.sqrt(mean_squared_error(true,pred))

def mean_absolute_error(true,pred):
    return np.mean(np.abs(pred - true))

def r2_score(true,pred):
    y_bar = np.mean(true)
    SSE = np.sum((pred - true)**2)
    SST = np.sum((true - y_bar)**2)
    return 1.-(SSE/SST)

def adj_r2(rsquare, num_data, num_features):
    temp = (1-rsquare)*(num_data-1)
    temp = temp/(num_data-num_features-1)
    temp = 1 - temp
    return temp

def standard_error_estimate(true,pred,num_data):
    SSE = np.sum((pred - true)**2)
    return np.sqrt(SSE/(num_data-2))

def plot_model_results(X,y,ypred):
    plt.style.use('seaborn')
    plt.scatter(ypred,y,s=55)
    plt.plot([min(y),max(y)],[min(y),max(y)],'r-')
    plt.xlabel('Predicted')
    plt.ylabel('True');

def model_score(true, pred, X, verbose=0):
    r2 = r2_score(true,pred)
    adjr2 = adj_r2(r2,X.shape[0],X.shape[1])
    if verbose:
        plot_model_results(X,true,pred)
        print("Mean Squared Error: ", mean_squared_error(true,pred))
        print("Root Mean Squared Error: ", np.sqrt(mean_squared_error(true,pred)))
        print("Mean Absolute Error: ",mean_absolute_error(true,pred))
        print("R2: ", r2)
        print("Adj R2: ", adjr2)
        print("Standard Error of Estimate: ", standard_error_estimate(true,pred,X.shape[0]))
    return r2, adjr2

In [99]:
model_score(y_test, y_predicted, X_test)

(0.5286427275848886, 0.5279942906409597)